In [1]:
from nltk.tokenize.casual import TweetTokenizer
import warnings
  
warnings.filterwarnings(action = 'ignore')
  
import gensim
from gensim.models import Word2Vec
from tqdm import tqdm
import numpy as np
import pandas as pd
import re
import torch

In [2]:
#  Reads ‘alice.txt’ file
sent=[]
y=[]
with open('twitter-datasets/train_pos_full.txt',errors="ignore") as pos:
    for line in pos:
        line=line.replace("\n", " ")#Replaces escape character with space
        sent.append(line)
        y.append(1)
tweet_pos=sent.copy()
with open('twitter-datasets/train_neg_full.txt',errors="ignore") as neg:
    for line in neg:
        line=line.replace("\n", " ")#Replaces escape character with space
        sent.append(line)
        y.append(0)
with open('twitter-datasets/test_data.txt',errors="ignore") as neg:
    for line in neg:
        line=line.replace("\n", " ")#Replaces escape character with space
        line=re.sub(r'^\d*,', '', line)
        #line=re.sub(r'<url>', '', line)
        sent.append(line)

### loading the test_tweets

In [5]:
test_sent=[]
with open('twitter-datasets/test_data.txt',errors="ignore") as neg:
    for line in neg:
        line=line.replace("\n", " ")#Replaces escape character with space
        line=re.sub(r'^\d*,', '', line)
        #line=re.sub(r'<url>', '', line)
        test_sent.append(line)

In [6]:
print(len(test_sent))
print(type(sent))
test_sent[:1]

10000
<class 'list'>


['sea doo pro sea scooter ( sports with the portable sea-doo seascootersave air , stay longer in the water and ... <url> ']

In [7]:
print(len(y))

2500000


In [7]:
from help_wordvec import tokenisation

In [8]:
data=tokenisation(sent)

100%|█████████████████████████████████████████████████████████████████████| 2510000/2510000 [03:01<00:00, 13842.32it/s]


In [9]:
len(data)

2510000

In [10]:
test_data=tokenisation(test_sent)

100%|█████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 15795.52it/s]


In [11]:
print(len(test_data))
test_data[1]

10000


['<user>',
 'shucks',
 'well',
 'i',
 'work',
 'all',
 'week',
 'so',
 'now',
 'i',
 "can't",
 'come',
 'cheer',
 'you',
 'on',
 '!',
 'oh',
 'and',
 'put',
 'those',
 'batteries',
 'in',
 'your',
 'calculator',
 '!',
 '!',
 '!']

In [12]:
 # Create CBOW model
model1 = gensim.models.Word2Vec(data, min_count = 1, 
                              vector_size = 200, window = 5,epochs=10)
  

In [13]:
print(model1.wv['mother'].shape)
print(model1.wv['mother'])
model1.vector_size

(200,)
[-1.1267532e+00  2.0424154e+00  5.6253474e-02 -5.7502318e-02
  6.0615790e-01  1.0464619e+00 -1.1726331e+00  1.3233373e+00
 -1.3385736e+00  1.0588614e+00  1.9110162e+00 -2.3552212e-01
  4.1116431e-02 -2.4932945e+00  4.5595059e-01  2.1658373e+00
 -1.3586603e+00  1.0816740e+00 -4.6649355e-02  3.3074087e-01
 -5.8651942e-01  3.3023441e-01 -1.6679669e-02 -7.9627335e-01
 -4.2344838e-01  1.0586134e+00  6.6553891e-01  6.8259996e-01
  2.4531287e-01  1.1568649e+00 -3.2750168e+00  2.3219988e-01
  3.1105421e+00 -2.3448753e-01 -1.8756492e+00 -1.5655328e-01
  1.5983626e+00  4.3604136e+00  1.8746079e+00 -5.9475195e-01
 -5.2616352e-01  1.5364137e+00 -1.1431412e+00  7.9961354e-01
 -3.0530274e+00  5.8786774e-01 -5.2212667e-01 -2.3319021e-01
  4.2945853e-01  1.0320014e+00  1.0852656e+00 -6.6833310e-02
 -7.2479911e-02 -2.3506124e+00  2.1612411e+00  5.0549775e-01
 -5.1804167e-01  2.6765940e-01  9.9062318e-01  3.5376284e-01
  4.1665480e-01  1.2633592e+00  1.9703778e+00  4.2566162e-02
  2.3353164e+00 -

200

### Making the feature matrix from the embeded vectors 

In [14]:
from help_wordvec import feature

In [15]:
test_tx=feature(test_data,model1)
print(test_tx.shape)

100%|█████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 17806.38it/s]

(10000, 200)


In [16]:
np.save('test_tx',test_tx)

In [17]:
tx=feature(data,model1)
print(tx.shape)

100%|█████████████████████████████████████████████████████████████████████| 2510000/2510000 [02:39<00:00, 15693.56it/s]


(2510000, 200)


In [18]:
np.save('tx_full',tx)

In [3]:
tx=np.load('tx_full.npy')

In [4]:
tx=tx[:2500000,:]
tx.shape

(2500000, 200)

In [5]:
from help_wordvec import split_data


In [12]:
y=np.array(y).reshape(-1,1)
print(y.shape)
[y_tr,y_te,x_tr,x_te]=split_data(tx,y,0.9,20)

(2500000, 1)


In [13]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score

# SGD

In [14]:
clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=10000,verbose=True)
clf.fit(x_tr, y_tr)
y_pred=clf.predict(x_te)
#scores = cross_val_score(clf, tx, y,scoring='accuracy', cv=4)

-- Epoch 1
Norm: 5.86, NNZs: 200, Bias: 1.983053, T: 2250000, Avg. loss: 1.062773
Total training time: 17.69 seconds.
-- Epoch 2
Norm: 5.43, NNZs: 200, Bias: 1.871007, T: 4500000, Avg. loss: 0.564499
Total training time: 19.34 seconds.
-- Epoch 3
Norm: 5.28, NNZs: 200, Bias: 1.706813, T: 6750000, Avg. loss: 0.546612
Total training time: 20.75 seconds.
-- Epoch 4
Norm: 5.22, NNZs: 200, Bias: 1.718150, T: 9000000, Avg. loss: 0.539463
Total training time: 22.16 seconds.
-- Epoch 5
Norm: 5.19, NNZs: 200, Bias: 1.713742, T: 11250000, Avg. loss: 0.535431
Total training time: 23.57 seconds.
-- Epoch 6
Norm: 5.16, NNZs: 200, Bias: 1.709556, T: 13500000, Avg. loss: 0.533006
Total training time: 24.98 seconds.
-- Epoch 7
Norm: 5.11, NNZs: 200, Bias: 1.726489, T: 15750000, Avg. loss: 0.531283
Total training time: 26.39 seconds.
-- Epoch 8
Norm: 5.09, NNZs: 200, Bias: 1.715938, T: 18000000, Avg. loss: 0.529889
Total training time: 27.79 seconds.
-- Epoch 9
Norm: 5.10, NNZs: 200, Bias: 1.694411, T:

In [15]:
y_pred.shape
y_te.shape

(250000, 1)

In [16]:
#print('Accuracy = '+str(scores.mean()))
acc=np.sum(y_pred==y_te.T)/len(y_te)
print('accuarcy = '+str(acc))

accuarcy = 0.774524


# KNN

In [15]:
from sklearn.neighbors import KNeighborsClassifier

In [1]:
from help_wordvec import take_part

In [22]:
y=np.array(y).reshape(-1,1)
tx_small,y_small=take_part(tx,y,1000000,20)
[y_tr,y_te,x_tr,x_te]=split_data(tx_small, y_small, 0.99, 20)

In [23]:
tx_small.shape

(1000000, 200)

In [24]:
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(x_tr, y_tr)

KNeighborsClassifier()

In [25]:
x_te.shape

(10000, 200)

In [26]:
y_pred=neigh.predict(x_te)
#scores = cross_val_score(neigh, tx, y,scoring='accuracy', cv=4)

In [27]:
#print('Accuracy = '+str(scores.mean()))
acc=np.sum(y_pred==y_te.T)/len(y_te)
print('accuarcy = '+str(acc))

accuarcy = 0.7752


# SVM

In [24]:
from sklearn.svm import SVC

In [25]:
y=np.array(y).reshape(-1,1)
tx_small,y_small=take_part(tx,y,100000,20)
[y_tr,y_te,x_tr,x_te]=split_data(tx_small, y_small, 0.9, 20)

In [26]:
tx_small.shape

(100000, 200)

In [27]:
SVmod=SVC(degree=4)
SVmod.fit(x_tr, y_tr)

SVC(degree=4)

In [28]:
y_pred=SVmod.predict(x_te)
#scores = cross_val_score(SVmod, tx, y,scoring='accuracy', cv=4)

In [29]:
acc=np.sum(y_pred==y_te.T)/len(y_te)
print('accuarcy = '+str(acc))
#print('Accuracy = '+str(scores.mean()))

accuarcy = 0.8174


# NN

In [75]:
from sklearn.neural_network import MLPClassifier

In [80]:
y=np.array(y).reshape(-1,1)
tx_small,y_small=take_part(tx,y,100000,20)
[y_tr,y_te,x_tr,x_te]=split_data(tx_small, y_small, 0.9, 20)

In [81]:
NN = MLPClassifier(solver='lbfgs', alpha=1e-5,
                     hidden_layer_sizes=(20), random_state=1, activation='relu', max_iter=100,verbose=True)
NN.fit(x_tr, y_tr)


MLPClassifier(alpha=1e-05, hidden_layer_sizes=20, max_iter=100, random_state=1,
              solver='lbfgs', verbose=True)

In [82]:
y_pred=NN.predict(x_te)
#scores = cross_val_score(NN, tx, y,scoring='accuracy', cv=4)

In [83]:
acc=np.sum(y_pred==y_te.T)/len(y_te)
print('accuarcy = '+str(acc))
#print('Accuracy = '+str(scores.mean()))

accuarcy = 0.7981


# NN not sklearn

In [29]:
tx=np.load('tx_full.npy')
tx=tx[:2500000,:]
tx.shape

(2500000, 200)

In [30]:
tx.shape

(2500000, 200)

In [31]:
y=np.array(y,dtype=np.float32).reshape(-1,1)
#tx_small,y_small=take_part(tx,y,1000000,20)
[y_tr,y_te,x_tr,x_te]=split_data(tx, y, 0.9, 20)
y_tr.shape

(2250000, 1)

In [32]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [33]:
class MLP(nn.Module):
    def __init__(self, embedding_dim,hidden_size):
        super(MLP, self).__init__()
        # TODO: Define parameters / layers of a multi-layered perceptron with one hidden layer
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(embedding_dim, hidden_size),
            nn.ReLU(), 
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1),
            nn.Sigmoid())
    
    def forward(self, x):
        # TODO: implement forward computation
        return self.linear_relu_stack(x)
#class MLP(nn.Module):
#    def __init__(self, hidden_size,embed_dim):
#        super(MLP, self).__init__()
#        # TODO: Define parameters / layers of a multi-layered perceptron with one hidden layer
#        ### SOLUTION
#        self.fc1 = nn.Linear(embed_dim,hidden_size)
#        self.activation_fn = nn.Sigmoid()
#        self.fc2 = nn.Linear(hidden_size,1) 
#    def forward(self, x):
#        # TODO: implement forward computation
#        out = self.fc2(self.activation_fn(self.fc1(x)))
#        return out

In [34]:
def train(features, labels, model, lossfunc, optimizer, num_epoch):
    for epoch in range(num_epoch):
        # TODO: Step 1 - create torch variables corresponding to features and labels
        inputs = torch.from_numpy(features)
        targets = torch.from_numpy(labels)
        # TODO: Step 2 - compute model predictions and loss
        ### SOLUTION
        outputs = model(inputs)
        loss = lossfunc(outputs, targets)
        # TODO: Step 3 - do a backward pass and a gradient update step
        optimizer.zero_grad()  
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print ('Epoch [%d/%d], Loss: %.4f' %(epoch+1, num_epoch, loss.item()))


In [35]:
hidden_size = 200
learning_rate = 0.01
model = MLP(embedding_dim=200,hidden_size = hidden_size)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
lossfunc = nn.MSELoss()

In [36]:
y_tr

array([[0.],
       [1.],
       [0.],
       ...,
       [1.],
       [0.],
       [1.]], dtype=float32)

In [37]:
train(features=x_tr,
      labels=y_tr,
      model=model,
      lossfunc=lossfunc,
      optimizer=optimizer,
      num_epoch=50)

Epoch [1/50], Loss: 0.2495
Epoch [11/50], Loss: 0.2046
Epoch [21/50], Loss: 0.1682
Epoch [31/50], Loss: 0.1528
Epoch [41/50], Loss: 0.1468


In [38]:
ytr_pred = model(torch.from_numpy(x_tr)).data.numpy()
ytr_pred[ytr_pred>=0.5]=1
ytr_pred[ytr_pred<0.5]=0

In [39]:
ytr_pred.shape
#y_tr.shape

(2250000, 1)

In [40]:
acc=np.sum(ytr_pred==y_tr)/len(y_tr)
print('train accuarcy = '+str(acc))

train accuarcy = 0.7908542222222222


In [41]:
yte_pred = model(torch.from_numpy(x_te)).data.numpy()
yte_pred[yte_pred>=0.5]=1
yte_pred[yte_pred<0.5]=0

In [42]:
acc=np.sum(yte_pred==y_te)/len(y_te)
print('test accuarcy = '+str(acc))

test accuarcy = 0.78978


In [43]:
torch.save(model, 'model_NN_fulldata')

# Making the prdictions

In [44]:
model2 = torch.load('model_NN_fulldata')
test_tx=np.load('test_tx.npy')
test_tx=np.float32(test_tx)
print(test_tx.shape)
print(type(test_tx))
test_tx.dtype

(10000, 200)
<class 'numpy.ndarray'>


dtype('float32')

In [53]:
tx=np.load('tx_full.npy')
tx.dtype

dtype('float32')

### Testing if the saveing of the model have worked

In [53]:
ytr_pred = model2(torch.from_numpy(x_tr)).data.numpy()
ytr_pred[ytr_pred>=0.5]=1
ytr_pred[ytr_pred<0.5]=0

In [54]:
acc=np.sum(ytr_pred==y_tr)/len(y_tr)
print('train accuarcy = '+str(acc))

train accuarcy = 0.7908542222222222


In [55]:
yte_pred = model2(torch.from_numpy(x_te)).data.numpy()
yte_pred[yte_pred>=0.5]=1
yte_pred[yte_pred<0.5]=0

In [56]:
acc=np.sum(yte_pred==y_te)/len(y_te)
print('test accuarcy = '+str(acc))

test accuarcy = 0.78978


In [28]:
test_tx=np.load('test_tx.npy')

In [30]:
#y_predfin=SVmod.predict(test_tx)
y_predfin=neigh.predict(test_tx)
y_predfin

array([0, 0, 0, ..., 0, 1, 0])

In [31]:
y_predfin[y_predfin>=0.5]=1
y_predfin[y_predfin<0.5]=-1

## Does finally the predictions

In [32]:
import csv

In [33]:
def create_csv_submission(y_pred, name):
    """Function which creates a csv file for submission on AI crowd."""
    ids = [x for x in range(1, len(y_pred)+1)]
    with open(name, 'w') as csvfile:
        fieldnames = ['Id', 'Prediction']
        writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
        writer.writeheader()
        for r1, r2 in zip(ids, y_pred):
    
            writer.writerow({'Id':int(r1),'Prediction':int(r2)})

In [58]:
y_pred = model2(torch.from_numpy(test_tx)).data.numpy()
y_pred

array([[0.01130082],
       [0.45614445],
       [0.10727421],
       ...,
       [0.02670657],
       [0.7393625 ],
       [0.09938823]], dtype=float32)

In [59]:
y_pred.shape

(10000, 1)

In [60]:
y_pred[y_pred>=0.5]=1
y_pred[y_pred<0.5]=-1

In [34]:
create_csv_submission(y_predfin,'KNN_1000000')

In [120]:
y_pred[4589]

array([1.], dtype=float32)